In [ ]:
# 
# -- plot traces
# 

from exp_voroni import *

print (TRINITY_NPATCHS)
plot_voroni_one_video_one_user()
plot_voroni_one_video_all_users()
# plot_voroni_one_video_all_users(to_html=True)
# plot_voroni_one_user_one_video_with_matplot()

# todo
  - calculate tile fetch/user in planar and voroni
  - calculate tile fetch/user in voronoi

In [108]:
# 
# -- debug conversions
# 
TILES_WIDTH, TILES_HEIGHT = 6, 4
ONE_POINT = 0
traces = get_traces_one_video_one_user()
x = traces[ONE_POINT][0]
y = traces[ONE_POINT][1]
z = traces[ONE_POINT][2]
print((x, y, z))
print(cartesian_to_eulerian(x, y, z))
print(eulerian_to_cartesian(*cartesian_to_eulerian(x, y, z)))
print(from_position_to_tile(
    eulerian_in_range(*cartesian_to_eulerian(x, y, z)),
    TILES_WIDTH, TILES_HEIGHT)
)


(0.9953681038070946, 0.0050096442775475, -0.0960064653431297)
(0.005032913913804639, 1.6669508930452646)
[ 0.9953681   0.00500964 -0.09600647]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [2]:
# 
# -- heatmap rondon position_to_tile
# 
import numpy as np
import plotly.express as px
from head_motion_prediction.Utils import *
from exp_voroni import *
TILES_WIDTH, TILES_HEIGHT = 6, 4
# traces = get_traces_one_video_all_users()
# traces = get_traces_one_video_one_user()
traces = get_traces_one_video_one_user()[:1]
heatmap = []
for i in traces:
    heatmap.append(from_position_to_tile(eulerian_in_range(*cartesian_to_eulerian(i[0], i[1], i[2])),
                                         TILES_WIDTH, TILES_HEIGHT))
print("traces sum: " + str(traces.shape[0]))
print("reqs sum: " + str(np.sum(heatmap)))
px.imshow(np.sum(heatmap, axis=0))

traces sum: 1
reqs sum: 1


In [39]:
#
# -- heatmap VRCLient position_to_tile
#
import numpy as np
import plotly.express as px
from head_motion_prediction.Utils import *
TILES_WIDTH, TILES_HEIGHT = 6, 4
vp_deg = 110
vp_rad = vp_deg * np.pi / 180


def arc_dist(phi_1, theta_1, phi_2, theta_2):
    t_1 = np.cos(theta_1) * np.cos(theta_2)
    t_2 = np.sin(theta_1) * np.sin(theta_2) * np.cos(phi_1 - phi_2)
    return np.arccos(t_1 + t_2)

def from_position_to_tile_new(pos, t_hor, t_vert):
    d_hor = 2 * np.pi / t_hor
    d_vert = np.pi / t_vert
    distances = []
    for i in range(t_vert):
        for j in range(t_hor):
            phi_c = d_hor * (j + 0.5)
            theta_c = d_vert * (i + 0.5)
            a = arc_dist(pos[0], pos[1], phi_c, theta_c)
            distances.append(((i,j), a))
    return [x[0] for x in distances if x[1] <= vp_rad / 2]


heatmap = []
# traces = get_traces_one_video_all_users()
# traces = get_traces_one_video_one_user()
traces = get_traces_one_video_one_user()[:1]
for i in traces:
    tiles_in = from_position_to_tile_new(
        cartesian_to_eulerian(i[0], i[1], i[2]), TILES_WIDTH, TILES_HEIGHT)
    planar = np.zeros((TILES_HEIGHT, TILES_WIDTH))
    for x in tiles_in:
        planar[x[0]][x[1]] = 1
    heatmap.append(planar)
print("traces sum: " + str(traces.shape[0]))
print("reqs sum: " + str(np.sum(heatmap)))
px.imshow(np.sum(heatmap, axis=0))


traces sum: 1
reqs sum: 4.0


In [15]:
# 
# -- np.array vs np.meshgrid
# 
import numpy as np
TILES_WIDTH, TILES_HEIGHT = 6, 4

heatmap = np.zeros((TILES_HEIGHT, TILES_WIDTH))
for i in range(TILES_HEIGHT):
    for j in range(TILES_WIDTH):
        if i == j:
            heatmap[i][j] = 1
print(heatmap)

# heatmap = np.meshgrid(np.zeros(TILES_HEIGHT), np.zeros(TILES_WIDTH))
# print(heatmap)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [ ]:
#
# -- from_position_to_tile_voroni
#
#
# -- heatmap VRCLient position_to_tile
#
import numpy as np
import plotly.express as px
from head_motion_prediction.Utils import *
TILES_WIDTH, TILES_HEIGHT = 6, 4
vp_deg = 110
vp_rad = vp_deg * np.pi / 180


def arc_dist(phi_1, theta_1, phi_2, theta_2):
    t_1 = np.cos(theta_1) * np.cos(theta_2)
    t_2 = np.sin(theta_1) * np.sin(theta_2) * np.cos(phi_1 - phi_2)
    return np.arccos(t_1 + t_2)


def cart_to_spher(x, y, z):
    phi = np.arctan2(y, x)
    if phi < 0:
        phi += 2 * np.pi
    theta = np.arccos(z)
    return phi, theta


def from_position_to_tile_voroni(pos):
    radius = 1
    center = np.array([0, 0, 0])
    sv = SphericalVoronoi(TRINITY_POINTS, radius, center)
    distances = []

    for v in sv.vertices:
        phi_c, theta_c = cart_to_spher(v[0], v[1], v[2])
        a = arc_dist(pos[0], pos[1], phi_c, theta_c)
        distances.append(((i, j), a))

    return [x[0] for x in distances if x[1] <= vp_rad / 2]


heatmap = []
# traces = get_traces_one_video_all_users()
# traces = get_traces_one_video_one_user()
traces = get_traces_one_video_one_user()[:1]
for i in traces:
    tiles_in = from_position_to_tile_voroni(
        cartesian_to_eulerian(i[0], i[1], i[2]))
    planar = np.zeros((TILES_HEIGHT, TILES_WIDTH))
    for x in tiles_in:
        planar[x[0]][x[1]] = 1
    heatmap.append(planar)
print("traces sum: " + str(traces.shape[0]))
print("reqs sum: " + str(np.sum(heatmap)))
# px.imshow(np.sum(heatmap, axis=0))
